# Libs imports and configs

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

# Checking data completness

In [6]:
def check_columns():
    first_file_cols = None

    for folder in os.listdir('./data'):
        for file in os.listdir(f'./data/{folder}'):
            data = pd.read_csv(f'./data/{folder}/{file}')
            num_of_cols = data.shape[1]

            if first_file_cols is None:
                first_file_cols = num_of_cols
            elif first_file_cols != num_of_cols:
                return 'There is some mismatch somewhere in csvs'
    
    return 'Number of columns are matching'

In [7]:
check_columns()

'Number of columns are matching'

In [8]:
data = pd.DataFrame()

for folder in os.listdir('./data'):
    for file in os.listdir(f'./data/{folder}'):
        df = pd.read_csv(f'./data/{folder}/{file}')
        data = pd.concat([data, df], ignore_index=True)

# Basic data description

In [9]:
data.shape

(25945, 66)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25945 entries, 0 to 25944
Data columns (total 66 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   timestamp                            25945 non-null  int64  
 1   date_GMT                             25945 non-null  object 
 2   status                               25945 non-null  object 
 3   attendance                           20594 non-null  float64
 4   home_team_name                       25945 non-null  object 
 5   away_team_name                       25945 non-null  object 
 6   referee                              16175 non-null  object 
 7   Game Week                            25184 non-null  float64
 8   Pre-Match PPG (Home)                 25945 non-null  float64
 9   Pre-Match PPG (Away)                 25945 non-null  float64
 10  home_ppg                             25945 non-null  float64
 11  away_ppg                    

# Setting first target variable - result of the game

In [11]:
data['result'] = np.where(data['home_team_goal_count'] == data['away_team_goal_count'], 0, np.where(data['home_team_goal_count'] > data['away_team_goal_count'], 1, 2))

In [13]:
#11,705 wins by home team, 7,611 wins by away team, 6,629 draws
data['result'].value_counts()

result
1    11705
2     7611
0     6629
Name: count, dtype: int64